### Price Analysis

In [1]:
import platform
from datetime import date

import dtale
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyodbc
import scipy.stats
import seaborn as sns
import sweetviz
from scipy.interpolate import UnivariateSpline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL

# import pandas_profiling


pd.set_option("display.max_columns", None)

In [2]:
import os

# Change the current working directory
os.chdir(
    "C:/Users/DeepankarSrigyan/OneDrive - Sykes Cottages Ltd/Desktop/Project/Base Pricing/Analysis/"
)

In [3]:
def fetch_SQLserver_data(sql_code, conn_string):
    """
    Uses pyodbc.connect and pandas.read_sql to return query results as dataframe.
    :param sql_code: string, query to be submitted
    :param conn_string: string, full connection details as string
    """
    connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_string})
    engine = create_engine(connection_url)
    with engine.connect() as conn:
        df = pd.read_sql(sql=text(sql_code), con=conn)
        conn.commit()
    print("Data Imported")
    return df


if platform.system() == "Windows":
    conn_CDM = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=CDM;trusted_connection=Yes"
    conn_l0 = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=Sykes_DW_Level0_Staging;trusted_connection=Yes"
    conn_l2 = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=Sykes_DW_Level2_ADM;trusted_connection=Yes"
    conn_BOX = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=BI-DATA-PROD-04;DATABASE=Box;trusted_connection=Yes"

    print("Connection Completed")

Connection Completed


In [9]:
PriceIssues = pd.read_excel('Price analysis base data.xlsx')
PriceIssues

,PropertyReference,Year,PricingIssues,Type,Reason,CheckDuplicate
0,3888,2024,YES,UNDER,UNDER,0.0
1,12256,2023,Yes,Pricing Decrease 5-10%,OVER,0.0
2,14722,2024,YES,UNDER,UNDER,0.0
3,24558,2024,YES,OVER,OVER,0.0
4,24650,2022,Yes,Pricing Decrease 10-20%,OVER,0.0
...,...,...,...,...,...,...
1844,1155233,2024,YES,OVER,OVER,0.0
1845,1155515,2024,YES,OVER,OVER,0.0
1846,1156306,2024,YES,OVER,OVER,0.0
1847,1156729,2024,YES,OVER,OVER,0.0


In [7]:
property = fetch_SQLserver_data(
"""
SELECT *
FROM CDM.propertymanagement.Property p
WHERE p.propertylivedate >= '2021-09-01'
    AND p.propertylivedate <= '2024-05-13'
    AND p.PropertyLiveDate IS NOT NULL
""",
    conn_CDM,
)
property

Data Imported


,PropertyID,PropertyVersionStartDateLocal,PropertyReference,PropertyReferenceType,ParkReference,ParkReferenceType,PropertyLeadID,BrandID,OwnerID,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,Latitude,Longitude,ParkName,PricingArea,PropertyName,PropertyStatus,PropertyType,Rating,RatingType,Sleeps,SykesTicks,isHoneyPot,isTestProperty,Northings,Eastings,LocationLongName,LocationType,County,Country,Location,WorkingRegion,PropertyLiveDate,AvailableFromDate,AvailableToDate,DateToldWithdrawingLocal,WithdrawalReason,WithdrawalWhatReason,WithdrawalWhyReason,StopSell,CommissionRate,RevenueManagementOption,ReinstatementDate,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,hasSpecialOffers,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual
0,55509,2024-04-10 23:01:49,926565,SykesPropertyID,SykesPark:0,SykesParkID,68832.0,1,106867.0,True,2,1,1,Friday,50.166862,-5.101134,None,HP Cornwall Coast,Maia House,Current,Flat,3,Ticks 1 to 5,2.0,3.0,True,False,34500.0,178500.0,"Penryn, Cornwall",Town,Cornwall,England,Penryn,Cornwall,2024-04-10,2024-04-10,None,None,Unknown,Unknown,Unknown,False,20.0,Partial Income Maximisation,2024-04-10,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,False,False,False,False,True,True,False,False,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False
1,222256,2023-01-26 02:59:20,1127224,SykesPropertyID,SykesPark:0,SykesParkID,245894.0,1,522047.0,False,0,2,2,Friday,53.279677,-4.616055,None,HP - Wales Anglesey Coastal,4 Y Bae,Current,Apartment,4,Ticks 1 to 5,4.0,4.0,True,False,379500.0,225500.0,"Trearddur, Isle of Anglesey",Other settlement,Isle of Anglesey,Wales,Trearddur,North Wales,2023-01-25,2023-01-27,None,None,Unknown,Unknown,Unknown,False,16.0,Partial Income Maximisation,None,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,False,True,False,False,True,True,True,False,True,False,False,False,True,False,True,True,False,False,False,False,True,True,False,True,False,False,True,False,False,False,False,False
2,222258,2023-09-05 02:59:27,1127242,SykesPropertyID,SykesPark:0,SykesParkID,245912.0,1,522059.0,False,0,1,4,Friday,53.338057,0.263404,None,Lincolnshire,80G,Lapsed,House,3,Ticks 1 to 5,8.0,3.0,False,False,385500.0,550500.0,"Mablethorpe, Lincolnshire",Town,Lincolnshire,England,Mablethorpe,East Anglia,2023-02-22,2023-02-22,2023-09-04,2023-05-03,Selling,Selling,Lack of Bookings,False,15.0,Super Income Maximisation,None,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,True,True,True,False,True,False,False,False,True,False,True,True,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False
3,222259,2023-06-29 22:54:51,1127249,SykesPropertyID,SykesPark:0,SykesParkID,245919.0,1,522061.0,True,2,2,3,Friday,51.322982,-2.321678,None,Bristol Region,Long Cottage,Current,Cottage,4,Ticks 1 to 5,6.0,4.0,False,False,158500.0,377500.0,"Hinton Charterhouse, Bath and North East Somerset",Other settlement,Bath and North East Somerset,England,Hinton Charterhouse,Somerset,2023-03-22,2023-03-31,None,None,Unknown,Unknown,Unknown,False,15.0,Super Income Maximisation,None,True,True,True,False,True,False,False,False,True,True,False,False,False,False,False,True,False,True,False,False,False,True,True,False,False,True,False,False,False,True

In [11]:
price=pd.merge(property,PriceIssues,on='PropertyReference',how='left')
price

,PropertyID,PropertyVersionStartDateLocal,PropertyReference,PropertyReferenceType,ParkReference,ParkReferenceType,PropertyLeadID,BrandID,OwnerID,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,Latitude,Longitude,ParkName,PricingArea,PropertyName,PropertyStatus,PropertyType,Rating,RatingType,Sleeps,SykesTicks,isHoneyPot,isTestProperty,Northings,Eastings,LocationLongName,LocationType,County,Country,Location,WorkingRegion,PropertyLiveDate,AvailableFromDate,AvailableToDate,DateToldWithdrawingLocal,WithdrawalReason,WithdrawalWhatReason,WithdrawalWhyReason,StopSell,CommissionRate,RevenueManagementOption,ReinstatementDate,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,hasSpecialOffers,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual,Year,PricingIssues,Type,Reason,CheckDuplicate
0,55509,2024-04-10 23:01:49,926565,SykesPropertyID,SykesPark:0,SykesParkID,68832.0,1,106867.0,True,2,1,1,Friday,50.166862,-5.101134,None,HP Cornwall Coast,Maia House,Current,Flat,3,Ticks 1 to 5,2.0,3.0,True,False,34500.0,178500.0,"Penryn, Cornwall",Town,Cornwall,England,Penryn,Cornwall,2024-04-10,2024-04-10,None,None,Unknown,Unknown,Unknown,False,20.0,Partial Income Maximisation,2024-04-10,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,False,False,False,False,True,True,False,False,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
1,222256,2023-01-26 02:59:20,1127224,SykesPropertyID,SykesPark:0,SykesParkID,245894.0,1,522047.0,False,0,2,2,Friday,53.279677,-4.616055,None,HP - Wales Anglesey Coastal,4 Y Bae,Current,Apartment,4,Ticks 1 to 5,4.0,4.0,True,False,379500.0,225500.0,"Trearddur, Isle of Anglesey",Other settlement,Isle of Anglesey,Wales,Trearddur,North Wales,2023-01-25,2023-01-27,None,None,Unknown,Unknown,Unknown,False,16.0,Partial Income Maximisation,None,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,False,True,False,False,True,True,True,False,True,False,False,False,True,False,True,True,False,False,False,False,True,True,False,True,False,False,True,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
2,222258,2023-09-05 02:59:27,1127242,SykesPropertyID,SykesPark:0,SykesParkID,245912.0,1,522059.0,False,0,1,4,Friday,53.338057,0.263404,None,Lincolnshire,80G,Lapsed,House,3,Ticks 1 to 5,8.0,3.0,False,False,385500.0,550500.0,"Mablethorpe, Lincolnshire",Town,Lincolnshire,England,Mablethorpe,East Anglia,2023-02-22,2023-02-22,2023-09-04,2023-05-03,Selling,Selling,Lack of Bookings,False,15.0,Super Income Maximisation,None,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,True,True,True,False,True,False,False,False,True,False,True,True,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,2023.0,Yes,Repricing Needed,Monitor,0.0
3,222259,2023-06-29 22:54:51,1127249,SykesPropertyID,SykesPark:0,SykesParkID,245919.0,1,522061.0,True,2,2,3,Friday,51.322982,-2.321678,None,Bristol Region,Long Cottage,Current,Cottage,4,Ticks 1 to 5,6.0,4.0,False,False,158500.0,377500.0,"Hinton Charterhouse, Bath and North East Somerset",Other settlement,Bath and North East Somerset,England,Hinton Charterhouse,Somerset,2023-03-22,2023-03-31,None,None,Unknown,Unknown,Unknown,False,15.0,Super Income Maximisation,None,True,True,True,False,True,False,False,Fal

In [14]:
price['Reason']=price['Reason'].fillna('NA')
price

,PropertyID,PropertyVersionStartDateLocal,PropertyReference,PropertyReferenceType,ParkReference,ParkReferenceType,PropertyLeadID,BrandID,OwnerID,AllowsPets,NumberOfPetsAllowed,Bathrooms,Bedrooms,ChangeOverDay,Latitude,Longitude,ParkName,PricingArea,PropertyName,PropertyStatus,PropertyType,Rating,RatingType,Sleeps,SykesTicks,isHoneyPot,isTestProperty,Northings,Eastings,LocationLongName,LocationType,County,Country,Location,WorkingRegion,PropertyLiveDate,AvailableFromDate,AvailableToDate,DateToldWithdrawingLocal,WithdrawalReason,WithdrawalWhatReason,WithdrawalWhyReason,StopSell,CommissionRate,RevenueManagementOption,ReinstatementDate,hasBroadband,hasDishwasher,hasEnclosedGarden,hasGames,hasGarden,hasHotTub,hasIndoorPool,hasLivestock,hasOffRoadParking,hasOpenFire,hasPoolTable,hasSauna,hasSpaOrGym,hasSpaJacuzzi,hasSwimmingPool,hasWashingMachine,isCoastal,hasCountryside,hasFishing,isIsolated,hasNationalPark,hasNationalTrust,isNearPub,isNearShop,isNearCanal,isNearCycling,isNearLake,isNearRailway,isNearRiver,isNearWalks,isNew,hasNoAnimals,hasShortBreaksAllowed,hasSpecialOffers,isChildFriendly,hasCotAvailable,isGroundFloor,hasLastMinuteBreakAllowed,isSmokeFree,isSmoking,hasCharacter,isFamily,isFarm,isInGroup,isLogCabin,isLuxury,isRomantic,isThatched,isUnusual,Year,PricingIssues,Type,Reason,CheckDuplicate
0,55509,2024-04-10 23:01:49,926565,SykesPropertyID,SykesPark:0,SykesParkID,68832.0,1,106867.0,True,2,1,1,Friday,50.166862,-5.101134,None,HP Cornwall Coast,Maia House,Current,Flat,3,Ticks 1 to 5,2.0,3.0,True,False,34500.0,178500.0,"Penryn, Cornwall",Town,Cornwall,England,Penryn,Cornwall,2024-04-10,2024-04-10,None,None,Unknown,Unknown,Unknown,False,20.0,Partial Income Maximisation,2024-04-10,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,False,False,False,False,True,True,False,False,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NA,NaN
1,222256,2023-01-26 02:59:20,1127224,SykesPropertyID,SykesPark:0,SykesParkID,245894.0,1,522047.0,False,0,2,2,Friday,53.279677,-4.616055,None,HP - Wales Anglesey Coastal,4 Y Bae,Current,Apartment,4,Ticks 1 to 5,4.0,4.0,True,False,379500.0,225500.0,"Trearddur, Isle of Anglesey",Other settlement,Isle of Anglesey,Wales,Trearddur,North Wales,2023-01-25,2023-01-27,None,None,Unknown,Unknown,Unknown,False,16.0,Partial Income Maximisation,None,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,False,True,False,False,True,True,True,False,True,False,False,False,True,False,True,True,False,False,False,False,True,True,False,True,False,False,True,False,False,False,False,False,NaN,NaN,NaN,NA,NaN
2,222258,2023-09-05 02:59:27,1127242,SykesPropertyID,SykesPark:0,SykesParkID,245912.0,1,522059.0,False,0,1,4,Friday,53.338057,0.263404,None,Lincolnshire,80G,Lapsed,House,3,Ticks 1 to 5,8.0,3.0,False,False,385500.0,550500.0,"Mablethorpe, Lincolnshire",Town,Lincolnshire,England,Mablethorpe,East Anglia,2023-02-22,2023-02-22,2023-09-04,2023-05-03,Selling,Selling,Lack of Bookings,False,15.0,Super Income Maximisation,None,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,True,True,True,False,True,False,False,False,True,False,True,True,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,2023.0,Yes,Repricing Needed,Monitor,0.0
3,222259,2023-06-29 22:54:51,1127249,SykesPropertyID,SykesPark:0,SykesParkID,245919.0,1,522061.0,True,2,2,3,Friday,51.322982,-2.321678,None,Bristol Region,Long Cottage,Current,Cottage,4,Ticks 1 to 5,6.0,4.0,False,False,158500.0,377500.0,"Hinton Charterhouse, Bath and North East Somerset",Other settlement,Bath and North East Somerset,England,Hinton Charterhouse,Somerset,2023-03-22,2023-03-31,None,None,Unknown,Unknown,Unknown,False,15.0,Super Income Maximisation,None,True,True,True,False,True,False,False,False

In [16]:
price.to_excel('price.xlsx')